In [3]:
import numpy as np
import pandas as pd
import requests
import json

In [4]:
raw_data_path = "../data/pek16.10.27departure.csv"
data_with_flightID_path = '../data/flightIDs/pek16.10.28depatureFlightID.csv'
flightID_path = "../data/flightIDs/pek16.10.27departureFlightID.csv"

In [5]:
arr_df =pd.read_csv(raw_data_path)

In [79]:
callsignG = arr_df.groupby("Callsign")

In [80]:
lastRecord = callsignG.last()

In [81]:
lastRecord = lastRecord.reset_index()

In [58]:
def getflightID(callsign):
    url1 = "https://www.flightradar24.com/v1/search/web/find?query=" + callsign + "&limit=18&type=schedule"
    r = requests.get(url1)
    flights = json.loads(r.text)["results"]
    res = ""
    for aFlight in flights:
        label = aFlight['label']
        words = label.split(' ')
        for word in words:
            if word == callsign:
                res = aFlight
                break
    if "id" in res:
        return res['id']
    else:
        return '-1'

In [59]:
lastRecord['FlightID'] = ""

In [60]:
%%time
lastRecord['FlightID'] = lastRecord.apply(lambda x: getflightID(x.Callsign), axis=1 )

CPU times: user 14 s, sys: 669 ms, total: 14.7 s
Wall time: 15min 33s


In [61]:
lastRecord.head()

,Callsign,trajID,Timestamp,Time,Position,Altitude,Speed,Direction,FlightID
0,3U8896,b718e54,1477629457,2016-10-28 12:37:37,"28.831833,104.550919",25600,428,199,3U8896
1,AAL186,b715fe6,1477666041,2016-10-28 22:47:21,"41.977055,-87.893028",0,2,84,AA186
2,AAL262,b71e9fe,1477661685,2016-10-28 21:34:45,"53.922958,163.93689",36975,515,60,AA262
3,AAR314,b7172ff,1477629965,2016-10-28 12:46:05,"35.185947,128.93898",0,0,50,OZ314
4,AAR316,b71eefc,1477652563,2016-10-28 19:02:43,"35.186043,128.940857",0,33,104,OZ316


In [62]:
flightIDMap  = lastRecord.drop(['Time','Position','Altitude','Speed','Direction'],axis=1)

In [63]:
flightIDMap.head()

,Callsign,trajID,Timestamp,FlightID
0,3U8896,b718e54,1477629457,3U8896
1,AAL186,b715fe6,1477666041,AA186
2,AAL262,b71e9fe,1477661685,AA262
3,AAR314,b7172ff,1477629965,OZ314
4,AAR316,b71eefc,1477652563,OZ316


In [64]:
flightIDMap.to_csv(flightID_path,index=False)

# remove illeagel Callsigns

In [102]:
illegalCallsigns = flightIDMap[flightIDMap['FlightID'] == "-1"]['Callsign'].tolist()

In [103]:
df = arr_df[np.logical_not(arr_df['Callsign'].isin(illegalCallsigns))]

In [104]:
df = df[np.logical_not(pd.isnull(df['Callsign']))]

# remove duplicate 

In [105]:
df.shape

(218182, 8)

In [106]:
df = df.drop_duplicates()

In [107]:
df.shape

(218182, 8)

# add flightID to raw data

In [108]:
flightIDMap2 = flightIDMap.set_index('Callsign')

In [109]:
callsignMap = flightIDMap2.to_dict('index')

In [110]:
df['FlightID'] = df.apply(lambda x: callsignMap[x.Callsign]['FlightID'],axis=1)

In [111]:
df.head()

,trajID,Timestamp,UTC,Callsign,Position,Altitude,Speed,Direction,FlightID
0,b6da94c,1477501879,2016-10-27 01:11:19,AHY68,"40.075333,116.591095",0,0,48,J268
1,b6da94c,1477501924,2016-10-27 01:12:04,AHY68,"40.075123,116.591141",0,10,120,J268
2,b6da94c,1477501951,2016-10-27 01:12:31,AHY68,"40.074841,116.591187",0,11,172,J268
3,b6da94c,1477501975,2016-10-27 01:12:55,AHY68,"40.074612,116.5914",0,10,132,J268
4,b6da94c,1477501993,2016-10-27 01:13:13,AHY68,"40.074509,116.591614",0,2,109,J268


In [112]:
df.to_csv(data_with_flightID_path, index=False)